In [13]:
#Import packages
from polygon.rest import RESTClient
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import numpy as np

#Import API keys from file
polygon_api_key = open("polygon_io_api_key.txt").read()
alphavantage_api_key = open("alphavantage_api_key.txt").read()

#Ticker
ticker = "afrm".upper()

# Calls the API to get historical stock data
stock_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={alphavantage_api_key}'
stock_r = requests.get(stock_url)
stock_data = stock_r.json()

earnings_url = f"https://api.polygon.io/benzinga/v1/earnings?ticker={ticker}&limit=100&sort=date.desc&apiKey={polygon_api_key}&date_status=confirmed"
earnings_r = requests.get(earnings_url)
earnings_data = earnings_r.json()

In [ ]:
##############
# Stock Data #
##############

# Extract the time series data
time_series = stock_data['Time Series (Daily)']

# Convert the time series data into a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Convert the columns to numeric
df = df.apply(pd.to_numeric)

df = df.reset_index().rename(columns={'index': 'tradingDate'})

In [34]:

#Earnings
df_len = []
for i in range(len(earnings_data["results"])):
    df_len.append(i)

benzinga_earnings = pd.DataFrame(earnings_data["results"], index=df_len)
df_column_drop = ['actual_eps','estimated_eps','previous_eps','eps_surprise_percent','revenue_surprise_percent','currency','date_status','benzinga_id','importance','company_name','estimated_revenue','previous_revenue','last_updated','eps_surprise','eps_method','actual_revenue','revenue_surprise','revenue_method',]
benzinga_earnings.drop(df_column_drop, axis='columns', inplace=True)

noon = dt.time(12,0,0)
market_periods = []

benzinga_earnings.loc[:, "time"] = pd.to_datetime(benzinga_earnings["time"], format='%H:%M:%S').dt.time

for time in benzinga_earnings["time"]:
    if time >= noon:
        market_periods.append('after-market')
    else:
        market_periods.append('pre-market')

benzinga_earnings.loc[:,'market_period'] = market_periods
benzinga_earnings.drop('time',axis="columns", inplace=True)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['market_period'] == 'after-market':
       return df[df["tradingDate"] > row['date']].min(axis=0)["tradingDate"]
   elif row['market_period'] == 'pre-market':
       return df[df["tradingDate"] < row['date']].max(axis=0)["tradingDate"]
   else:
       return row['date']

benzinga_earnings['date'] = pd.to_datetime(benzinga_earnings['date'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
benzinga_earnings['adjustedDate'] = benzinga_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(benzinga_earnings, df, left_on='date', right_on="tradingDate", how='left')
columns_to_drop = ['tradingDate','5. volume']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   '1. open': 'report open',
   '2. high': 'report high',
   '3. low': 'report low',
   '4. close': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="tradingDate", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   '1. open': 'adjusted open',
   '2. high': 'adjusted high',
   '3. low': 'adjusted low',
   '4. close': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df.sort_values(by="percent_diff", ascending=False)

,fiscal_period,fiscal_year,ticker,date,market_period,adjustedDate,report open,report high,report low,report close,adjusted open,adjusted high,adjusted low,adjusted close,diff,percent_diff
13,Q3,2022,AFRM,2022-05-12,after-market,2022-05-13,14.140,19.3700,13.6401,18.04,24.8300,25.3200,21.2100,23.71,11.68,0.856
16,Q4,2021,AFRM,2021-09-09,after-market,2021-09-10,88.000,92.2900,86.7600,92.06,111.1000,126.4600,105.6000,123.70,39.70,0.458
4,Q4,2024,AFRM,2024-08-28,after-market,2024-08-29,32.230,32.5100,30.4601,31.58,39.1100,42.8700,37.5200,41.66,12.41,0.407
7,Q1,2024,AFRM,2023-11-08,after-market,2023-11-09,21.510,22.1897,20.2000,21.76,24.9700,27.1566,24.4300,24.86,6.96,0.344
8,Q4,2023,AFRM,2023-08-24,after-market,2023-08-25,14.550,14.5800,13.7400,13.81,15.0900,18.3200,15.0000,17.79,4.58,0.333
2,Q2,2025,AFRM,2025-02-06,after-market,2025-02-07,63.120,63.3599,60.5600,61.75,69.9000,76.8800,68.3000,75.22,16.32,0.269
15,Q1,2022,AFRM,2021-11-10,after-market,2021-11-11,146.930,149.0000,132.1600,133.53,165.2501,166.5100,147.5000,151.83,34.35,0.260
5,Q3,2024,AFRM,2024-05-08,pre-market,2024-05-07,35.500,37.8800,31.0700,31.58,35.4450,36.5800,34.8600,34.90,3.02,0.087
9,Q3,2023,AFRM,2023-05-09,after-market,2023-05-10,11.600,12.7391,11.5400,12.30,12.1100,12.9650,11.6100,12.25,-1.13,-0.089
18,Q2,2021,AFRM,2021-02-11,after-market,2021-02-12,140.000,140.5000,131.1600,139.99,129.6397,131.6700,123.0100,125.89,-17.49,-0.124
